In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

In [ ]:
FTSUBV = np.zeros((6,1))
FTSUBV[0] = XNR[2*slackidx] - Vslack[0].real
FTSUBV[1] = XNR[2*slackidx+1] - Vslack[0].imag
FTSUBV[2] = XNR[2*nnode+2*slackidx] - Vslack[1].real
FTSUBV[3] = XNR[2*nnode+2*slackidx+1] - Vslack[1].imag
FTSUBV[4] = XNR[4*nnode+2*slackidx] - Vslack[2].real
FTSUBV[5] = XNR[4*nnode+2*slackidx+1] - Vslack[2].imag

In [2]:
dss.run_command('Redirect 05node_singlephase_balanced_oscillation_03.dss')
dss.Solution.Solve()

X = np.array([])
line_names = dss.Lines.AllNames()


for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        source_voltage = dss.Bus.PuVoltage()
    else:     
        X_temp = np.zeros(6)
        pu_temp = dss.Bus.PuVoltage()
        X_temp[:len(pu_temp)] = pu_temp
        X = np.append(X, X_temp)




[ 0.97898101 -0.03607562  0.          0.          0.          0.
  0.97803655 -0.0363734   0.          0.          0.          0.
  0.97661911 -0.03682004  0.          0.          0.          0.
  0.97614662 -0.0369689   0.          0.          0.          0.
  0.97614638 -0.03696892  0.          0.          0.          0.
  0.97567364 -0.03711777  0.          0.          0.          0.        ]


### XNR

In [25]:
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

I0 = np.zeros([3, nline])
s = np.array([])
g_t = np.array([])
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    v_temp = dss.Bus.PuVoltage()
    z = np.zeros(6)
    z[:len(v_temp)] = np.array([v_temp])
    s = np.append(s, z)
V_all = np.reshape(s, (int(len(s)/6), 6)).T
slack = V_all[:,:1]
V0 = V_all[:, 1:]

i = np.array([])
for k2 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k2]) 
    i_temp = dss.CktElement.Currents()
    z = np.zeros(6)
    z[:len(i_temp)] = i_temp
    i = np.append(i, z)
i = np.reshape(i, (6, len(i) // 6))

XNR = np.zeros((2*3*(nnode+nline),1))
print(XNR.shape)
for ph in range(0,3):
    for k1 in range(0,nnode):
     
        XNR[2*ph*nnode + 2*k1] = V_all[ph * 2 , k1]
        XNR[2*ph*nnode + 2*k1+1] = V_all[(ph * 2) +1,k1]

for ph in range(0,3):
    for k1 in range(0,nline):
        XNR[(2*3*nnode) + 2*ph*nline + 2*k1] = i[ph * 2,k1]
        XNR[(2*3*nnode) + 2*ph*nline + (2*k1)+1] = i[(ph * 2) + 1,k1]
#print(XNR)

slackidx = 0
FTSUBV = np.zeros((6,1))
FTSUBV[0] = XNR[2*slackidx] - slack[0]
FTSUBV[1] = XNR[2*slackidx+1] - slack[1]
FTSUBV[2] = XNR[2*nnode+2*slackidx] - slack[2]
FTSUBV[3] = XNR[2*nnode+2*slackidx+1] - slack[3]
FTSUBV[4] = XNR[4*nnode+2*slackidx] - slack[4]
FTSUBV[5] = XNR[4*nnode+2*slackidx+1] - slack[5]
print(FTSUBV)




(72, 1)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [22]:
slack

array([[ 9.99996847e-01],
       [-5.13569111e-06],
       [-4.99999967e-01],
       [-8.66025072e-01],
       [-4.99999968e-01],
       [ 8.66025735e-01]])

### Slack Bus

$$ \delta_{SBRE,m} = A_{m}^{\phi} - \Re \left\{ V_{sl}^{\phi} \right\} \\
\delta_{SBIM,m} = B_{m}^{\phi} - \Im \left\{ V_{sl}^{\phi} \right\} $$

In [ ]:
#G_Slack
G_slack = np.zeros(nnode * 2 * 3) #nnode * 2parts *3phases

real_slack = source_voltage[::2] 
imag_slack = source_voltage[1::2]

counter = 0
for n in range(0, len(AB), 2):
    for j in range(0, len(real_slack)):
        G_slack[counter] = AB[n] - real_slack[j]
        G_slack[counter + 1] = AB[n+1] - imag_slack[j]
        counter += 2
        
print(G_slack)


In [ ]:
#b_slack
b_slack = np.zeros(G_slack.size)


### KVL

$$ \begin{align}
\delta_{KVLRE,mn}^{\phi} = A_{m}^{\phi} - A_{n}^{\phi} - \sum_{\psi} r_{mn}^{\phi \psi} C_{mn}^{\psi} - x_{mn}^{\phi \psi} D_{mn}^{\psi}  \\
\delta_{KVLIM,mn}^{\phi} = B_{m}^{\phi} - B_{n}^{\phi} - \sum_{\psi} r_{mn}^{\phi \psi} D_{mn}^{\psi} + x_{mn}^{\phi \psi} C_{mn}^{\psi}
\end{align}
$$

In [ ]:
#GKVL
G_KVL = np.zeros((len(line_names)))

C_mn = np.array([])
D_mn = np.array([])
for k2 in range(len(dss.Lines.AllNames())):
   
    dss.Lines.Name(dss.Lines.AllNames()[k2]) #set the line
    bus1 = dss.Lines.Bus1() #get start bus
    bus2 = dss.Lines.Bus2() #get end bus
    

    dss.Circuit.SetActiveBus(bus1) #set active bus to bus 1
    linecode = dss.Lines.LineCode() #get the linecode
    dss.LineCodes.Name(linecode) #set the linecode
    xmat = dss.LineCodes.Xmatrix() #get the xmat
    rmat = dss.LineCodes.Rmatrix() #get the rmat
    volts = dss.Bus.PuVoltage() #get bus1's puVoltage
    A_m = volts[0] #split into re/im parts
    B_m = volts[1]
    
    for i in range(0, len(dss.CktElement.Currents()), 2): #get the currents of the line
        C_mn = np.append(C_mn, dss.CktElement.Currents()[i])
        D_mn = np.append(D_mn, dss.CktElement.Currents()[i+1])

    dss.Circuit.SetActiveBus(bus2)  #set active bus to bus 2
    volts = dss.Bus.PuVoltage() #get bus2's puVoltage
    A_n = volts[0] #split into re/im parts
    B_n = volts[1]
  
    re_residual = 0 #calculate the residuals
    im_residual = 0
    
    for i in range(len(C_mn)): #compute the summation part of the residual
        re_sum = re_residual + (rmat * C_mn + xmat * D_mn)
        im_sum = im_residual + (rmat * D_mn + xmat * C_mn)
        
    re_residual = A_m - A_n - (re_sum) #subtract real/imag volt parts from current
    im_residual = B_m - B_n - (im_sum)
    G_KVL = np.append(G_KVL, [re_residual, im_residual]) #add these residuals to G_KVL
    
    
    #Is G_KVL the right shape?
    #Am I properly calculating the summation? 
    #Does this work for 3 phase stuff? And what if the resistance is not just a scalar... ? 
print(G_KVL) 


In [ ]:
b_kvl = np.zeros(len(G_KVL))

### KCL

$$ u_{m}^{\phi}, c_{m}^{\phi},  v_{m}^{\phi} $$

In [ ]:
#b_k
cap_names = dss.CapControls.AllNames() #i don't see a method to get capacitance
active_power = np.array([])
reactive_power = np.array([])

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    Sk = dss.CktElement.Powers()
  
    active_power = np.append(active_power, Sk[0])
    reactive_power = np.append(reactive_power, Sk[1])
    
print(active_power)
print(reactive_power)


$$\sum_{l:(l,m) \in \mathcal{E}} \left( A_{m}^{\phi} C_{lm}^{\phi} + B_{m}^{\phi} D_{lm}^{\phi} \right)
-
\sum_{n:(m,n) \in \mathcal{E}} \left( A_{m}^{\phi} C_{mn}^{\phi} + B_{m}^{\phi} D_{mn}^{\phi} \right)$$

In [ ]:
def linelist(busname):
    print(busname)
    in_lines = np.array([])
    out_lines = np.array([])
    for k in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k])
        if busname in dss.Lines.Bus1():
            out_lines = np.append(out_lines, dss.Lines.AllNames()[k])
        elif busname in dss.Lines.Bus2():
            in_lines = np.append(in_lines,dss.Lines.AllNames()[k])
    return in_lines,out_lines
        
il , ol = linelist(dss.Circuit.AllBusNames()[3])
print(il)
print(ol)

In [ ]:
#the summation part of the KCL residual
G_kcl = np.array([])

for k2 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2]) #set active bus to bus 1
    #linelist = dss.Bus.LineList()
    
    volts = dss.Bus.PuVoltage() #get bus1's puVoltage
    A_m = volts[0] #split into re/im parts
    B_m = volts[1]
    
    #get in and out lines of bus[k2]
    in_lines, out_lines = linelist(dss.Circuit.AllBusNames()[k2])
    
    C_lm = np.array([]) #get incoming current re/im parts
    D_lm = np.array([])
    
    for i in range(len(in_lines)):
        try: 
            dss.Lines.Name(in_lines[i])
            v_i = dss.CktElement.Currents()
            for n in range(0, len(v), 2):
                C_lm = np.append(C_lm, v_i[n])
                D_lm = np.append(D_lm, v_i[n+1])
            print(v)
        except:
            pass
        
    C_mn= np.array([]) #get outgoing current re/im parts
    D_mn = np.array([])
    
    for i in range(len(out_lines)):
        try: 
            dss.Lines.Name(out_lines[i])
            v_o = dss.CktElement.Currents()
            for n in range(0, len(v), 2):
                C_mn = np.append(C_mn, v_o[n])
                D_mn = np.append(D_mn, v_o[n+1])
            print(v)
        except:
            pass


    re1 = 0
    im1 = 0
    re2 = 0
    im2 = 0
    
    for i in range(len(C_lm)): #first part of real and imaginary resid
        re1 = re1 + (A_m * C_lm[i] + B_m * D_lm[i])
        im1 = im1 + (-A_m * D_lm[i] + B_m * C_lm[i]) 
    for j in range(len(C_mn)): #second part of real and imaginary resid 
        re2 = re2 + (A_m * C_mn[i] + B_m*D_mn[i])
        im2 = im2 + (-A_m * D_mn[i] + B_m * C_mn[i])
    
    re_resid = re1 - re2
    im_resid = im1 - im2
    
    G_kcl = np.append(G_kcl, [re_resid, im_resid])

#     #Is G_kcL the right shape?
#     #Am I properly calculating the summation? 





$$ \beta_{I,m}^{\phi} \left( \left( A_{m}^{\phi} \right)^2 + \left( B_{m}^{\phi} \right)^{2} \right)^{\frac{1}{2}} + \beta_{Z,m}^{\phi} \left( \left( A_{m}^{\phi} \right)^2 + \left( B_{m}^{\phi} \right)^{2} \right) $$

In [ ]:
#g_k and H_k
#FTKCL

dA = XNR[idxAm] - A0
dB = XNR[idxBm] - B0

dX = np.array([dA[0], dB[0]]) #(1, 2)
dX_t = np.array([dA[0], dB[0]]).T #(2,1)

gradient_mag = np.array([A0 * ((A0**2+B0**2) ** (-1/2)), B0 * ((A0**2+B0**2) ** (-1/2))])
gradient_mag_sq = np.array([2 *A0, 2 * B0])

first_order_term_mag = ((A0**2+B0**2)**(1/2) + np.matmul(gradient_mag, dX_t)) 
first_order_term_mag_sq = (A0**2+B0**2) + np.matmul(gradient_mag_sq, np.array(dX_t))

hessian_mag_sq = np.array([2, 0], [0, 2])
hessian_mag = np.array([[-((A0**2)*(A0**2+B0**2)**(-3/2))+(A0**2+B0**2)**(-1/2), -A0*B0*(A0**2+B0**2)**(-3/2)],
                        [-A0*B0*(A0**2+B0**2)**(-3/2), -((B0**2)*(A0**2+B0**2)**(-3/2))+((A0**2+B0**2)**(-1/2))]])


second_order_term_mag_sq = (1/2) * np.matmul(dX_t,  2 * dX)
second_order_term_mag = (1/2) * np.matmul(dX_t, np.matmul(hessian_mag, dX))

#JKCL
first_order_term_mag = gradient_mag
first_order_term_mag_sq = gradient_mag_sq
second_order_term_mag = gradient_mag + (np.matmul(hessian_mag,dX).T)
second_order_term_mag_sq = gradient_mag_sq + (np.matmul(hessian_mag_sq, dX).T)




---

## Things of note
                    

In [ ]:
dA = 1
dB = 2
A0 = 5
B0 = 2
dX = np.array([dA, dB])
dX_t = np.array([dA, dB]).T
hess = np.array([[1,2], [3,4]])
np.matmul(dX_t, np.matmul(hess, dX))

In [ ]:
from opendssdirect.utils import Iterator
load_kW = [i() for i in Iterator(dss.Loads, 'kW')]
print(load_kW)

In [ ]:
#Pandas Functions
dss.utils.loads_to_dataframe()
# dss.utils.fuses_to_dataframe
# dss.utils.generators_to_dataframe
# dss.utils.isource_to_dataframe
#dss.utils.lines_to_dataframe()

# dss.utils.transformers_to_dataframe
#dss.utils.vsources_to_dataframe()
# dss.utils.xycurves_to_dataframe

